In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd


from mpl_toolkits.axes_grid1 import make_axes_locatable
import time




In [2]:
### Allows us to use dask to speed up some calculations ###
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44769,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.29 GiB
Comm: tcp://127.0.0.1:46055,Total threads: 4
Dashboard: http://127.0.0.1:39572/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:46040,


In [ ]:
era5_google = 'gcp-public-data-arco-era5/raw/date-variable-pressure_level'

In [ ]:
era5 = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/raw/date-variable-pressure_level/",
    chunks={'time': 48},
    consolidated=True,
)

In [3]:
data_dir = '/data/deluge/reanalysis/REANALYSIS/ERA5/3D/daily/hgt/'

In [4]:
ds = xr.open_mfdataset(data_dir+'*.nc',parallel=True, chunks={'latitude': 25, 'longitude': 25, 'time': -1})

In [ ]:
# lon=255, # lat=48.0

In [ ]:
data = ds.sel(latitude=48.0, longitude=255.0)['precip']

In [ ]:
ds

In [ ]:
OD_data = xr.open_dataset('OnsetDemise_ERA5.nc')

In [ ]:
OD_ts = OD_data.sel(latitude=36.0, longitude=264.0)

In [ ]:
onset_interval = np.empty(4)
demise_interval = np.empty(4)

def OD_ci(data):
    
    m = np.nanmean(data.values)
    s = np.nanstd(data.values)
    c = len(data.values)
    upper = (m + 1.96*s/np.sqrt(c))
    lower = (m - 1.96*s/np.sqrt(c))
    return np.array([m,s,upper,lower])

onset_interval = OD_ci(OD_ts['onset_doy'])
demise_interval = OD_ci(OD_ts['demise_doy'])


In [ ]:
onset_interval

In [ ]:
demise_interval

In [ ]:
#data is in meters
data = data * 1000.0

In [ ]:
### Creating the anomaly data
anomaly = data - data.mean()

In [ ]:
array_jday = anomaly.time.dt.dayofyear
anomaly = anomaly.assign_coords(jday=("time",array_jday.data))

In [ ]:
pd_anomaly = anomaly.rolling(time=60,center=True).sum().to_dataframe()

In [ ]:
plt.figure(figsize = (13, 8), dpi = 300)
sns.set_context("poster")

pd_anomaly['time'] = pd_anomaly.index
pd_anomaly['dayofyear'] = pd_anomaly['time'].dt.dayofyear
pd_anomaly['year'] = pd_anomaly['time'].dt.year
pd_anomaly['smooth_tp'] = signal.savgol_filter(pd_anomaly['precip'],31,3)

ax = sns.lineplot(data=pd_anomaly, x='dayofyear',y='smooth_tp',estimator=None, lw=1, alpha=.5, units='year',color='gray')
sns.lineplot(data=pd_anomaly, x='dayofyear',y='smooth_tp',palette='blue', ci='sd', err_style='band', ax=ax)

ax.axvline(onset_interval[0], color='#a5c7ac', linestyle='-', lw=3, label='Average Onset')

ax.axvline(demise_interval[0], color='#988558', linestyle='-', lw=3, label='Average Demise')
ax.axvspan(onset_interval[2],onset_interval[3],facecolor='#a5c7ac', alpha=0.5, linestyle='--', label='Onset 95% CI')
ax.axvline(onset_interval[2], color='#a5c7ac', linestyle='--', lw=1)
ax.axvline(onset_interval[3], color='#a5c7ac', linestyle='--', lw=1)

ax.axvspan(demise_interval[2],demise_interval[3],facecolor='#988558', alpha=0.5, linestyle='--',label='Demise 95% CI')
ax.axvline(demise_interval[2], color='#988558', linestyle='--', lw=1)
ax.axvline(demise_interval[3], color='#988558', linestyle='--', lw=1)
ax.set_ylabel('Daily P Anomaly (mm)')
ax.set_xlabel('Day of Year')
at = AnchoredText("b",
                      loc='upper left', prop=dict(size=20), frameon=True,)
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax.add_artist(at)

sns.set_context("talk")
ax.legend()


#ax.set_title('Southeast U.S.')


In [ ]:
wetseason_count = xr.open_dataarray('average_wetseasons_na.nc').sel(latitude=slice(60,20), longitude=slice((360.0-125),(360.0-65.0)))

In [ ]:
wetseason_count

In [ ]:
plt.figure(figsize = (27, 9), dpi = 300)
sns.set_context('poster')
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    facecolor='none')
map_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=45)
#cmap = mpl.cm.RdBu_r


ax1 = plt.subplot(1, 1,1, projection=map_proj)
p = wetseason_count.plot.contourf(ax=ax1,transform=ccrs.PlateCarree(), add_colorbar=False)

ax1.set_aspect('equal')
### Setting 1st plot parameters ###
ax1.coastlines(color='grey')
ax1.add_feature(cartopy.feature.BORDERS, color='grey')
#ax1.add_feature(cfeature.STATES, color='grey')
#ax1.set_xticks(np.arange(-180,181, 40))
#ax1.set_yticks(np.arange(-90,91,15))
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
#ax1.set_title('Standard Deviation of Pentads')
#at = AnchoredText("a",
#                      loc='upper left', prop=dict(size=20), frameon=True,)
#at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
#ax1.add_artist(at)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)
plt.colorbar(p, cax=cax)

In [ ]:
onset_interval[3]

In [ ]:
test = xr.open_dataset('wetseason.onset.era5.nc')

In [ ]:
test['onset_date'].sel(latitude=33,longitude=273).sel(year=2009)

In [ ]:
test2 = xr.open_dataset('wetseason.demise.era5.nc')


In [ ]:
test2['demise_date'].sel(latitude=33,longitude=273).sel(year=2009)